# <center>Read Keystroke Data and Build Model to Process</center>

In [1]:
import os
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import svm
import sklearn.cross_validation

input_path = "Dataset/Kevin and Maxion/DSL-StrongPasswordData.csv"
print input_path;

Dataset/Kevin and Maxion/DSL-StrongPasswordData.csv


## Import Raw Data and Extract Features. Pre-process the data 

In [31]:
file = open(input_path, "r");

raw_data = file.read();
print type(raw_data);

data_lines = np.array(raw_data.splitlines());
print data_lines[0];

data = np.array([ x.split(",") for x in data_lines]);
data = np.delete(data, 0, 0);
print data;
print type(data);
print data.shape;

<type 'str'>
subject,sessionIndex,rep,H.period,DD.period.t,UD.period.t,H.t,DD.t.i,UD.t.i,H.i,DD.i.e,UD.i.e,H.e,DD.e.five,UD.e.five,H.five,DD.five.Shift.r,UD.five.Shift.r,H.Shift.r,DD.Shift.r.o,UD.Shift.r.o,H.o,DD.o.a,UD.o.a,H.a,DD.a.n,UD.a.n,H.n,DD.n.l,UD.n.l,H.l,DD.l.Return,UD.l.Return,H.Return
[['s002' '1' '1' ..., '0.3509' '0.2171' '0.0742']
 ['s002' '1' '2' ..., '0.2756' '0.1917' '0.0747']
 ['s002' '1' '3' ..., '0.2847' '0.1762' '0.0945']
 ..., 
 ['s057' '8' '48' ..., '0.2017' '0.0983' '0.0905']
 ['s057' '8' '49' ..., '0.1917' '0.0938' '0.0931']
 ['s057' '8' '50' ..., '0.1993' '0.1186' '0.1018']]
<type 'numpy.ndarray'>
(20400L, 34L)


### Build train dataset and test dataset

In [60]:
# split data to train_data and test_data for cross validation

kf = sklearn.cross_validation.KFold(len(data), 5, True);
print kf;

for train_index, test_index in kf:
    train, test = data[train_index], data[test_index];
    
print train.shape;
print test.shape;

train_target = train[:, 0:1];
train_data = train[:, 3:];

test_target = test[:, 0:1];
test_data = test[:, 3:];

##change features' value from string to numerical(float)
train_data = train_data.astype(np.float);
test_data = test_data.astype(np.float);

print train_data;
print test_data;

n_classes = len(np.unique(train_target));
print (np.unique(train_target), np.unique(test_target));
print n_classes;

sklearn.cross_validation.KFold(n=20400, n_folds=5, shuffle=True, random_state=None)
(16320L, 34L)
(4080L, 34L)
[[ 0.1111  0.3451  0.234  ...,  0.2756  0.1917  0.0747]
 [ 0.1328  0.2072  0.0744 ...,  0.2847  0.1762  0.0945]
 [ 0.1291  0.2515  0.1224 ...,  0.3232  0.2387  0.0813]
 ..., 
 [ 0.0655  0.063  -0.0025 ...,  0.2206  0.0761  0.1198]
 [ 0.0923  0.1294  0.0371 ...,  0.1917  0.0938  0.0931]
 [ 0.0596  0.131   0.0714 ...,  0.1993  0.1186  0.1018]]
[[ 0.1491  0.3979  0.2488 ...,  0.3509  0.2171  0.0742]
 [ 0.0966  0.1797  0.0831 ...,  0.2868  0.1986  0.0634]
 [ 0.0903  0.1871  0.0968 ...,  0.3889  0.2701  0.0318]
 ..., 
 [ 0.0966  0.1126  0.016  ...,  0.1822  0.1046  0.1163]
 [ 0.0665  0.0678  0.0013 ...,  0.2047  0.0996  0.1105]
 [ 0.0939  0.1189  0.025  ...,  0.2017  0.0983  0.0905]]
(array(['s002', 's003', 's004', 's005', 's007', 's008', 's010', 's011',
       's012', 's013', 's015', 's016', 's017', 's018', 's019', 's020',
       's021', 's022', 's024', 's025', 's026', 's027', 's0

### Train the SVM model to analyze data

When training an SVM with the Radial Basis Function (RBF) kernel, two parameters must be considered: C and gamma. 
* The parameter C, common to all SVM kernels, trades off misclassification of training examples against simplicity of the decision surface. A low C makes the decision surface smooth, while a high C aims at classifying all training examples correctly.
* gamma defines how much influence a single training example has. The larger gamma is, the closer other examples must be to be affected.
* Proper choice of C and gamma is critical to the SVM’s performance. One is advised to use sklearn.grid_search.GridSearchCV with C and gamma spaced exponentially far apart to choose good values.

In [35]:
clf = svm.SVC(gamma=1, C=300.);
clf2 = svm.SVC(gamma=1, C=300., decision_function_shape='ovo');
## for now, gamma = 1, C = 300 is my best combination of those two parameters in this model

clf.fit(train_data, train_target);
clf2.fit(train_data, train_target);
# libsvm = svm.libsvm.fit(x, y);

print (clf, clf2);
print (clf.score(test_data, test_target), clf2.score(test_data, test_target));

for i in range(20):
    print(clf.predict(test_data[i]), test_target[i]);

    

D:\Anaconda2\lib\site-packages\sklearn\svm\base.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y_ = column_or_1d(y, warn=True)
D:\Anaconda2\lib\site-packages\sklearn\svm\base.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y_ = column_or_1d(y, warn=True)


SVC(C=300.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
SVC(C=300.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma=1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
0.880637254902
0.880637254902
(array(['s002'], 
      dtype='|S15'), array(['s002'], 
      dtype='|S15'))
(array(['s002'], 
      dtype='|S15'), array(['s002'], 
      dtype='|S15'))
(array(['s004'], 
      dtype='|S15'), array(['s002'], 
      dtype='|S15'))
(array(['s002'], 
      dtype='|S15'), array(['s002'], 
      dtype='|S15'))
(array(['s002'], 
      dtype='|S15'), array(['s002'], 
      dtype='|S15'))
(array(['s002'], 
      dtype='|S15'), array(['s002'], 
      dtype='|S15'))
(array(['s002'], 
      dtype='|S15'), array(['s002'], 
      dty

D:\Anaconda2\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
D:\Anaconda2\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
D:\Anaconda2\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
D:\Anaconda2\lib\site-packages\skl

### Use Neutral Network to analyze 
* In scikit-learn, the neural network model is Restricted Boltzmann machines (RBM). The features extracted by an RBM or a hierarchy of RBMs often give good results when fed into a linear classifier such as a linear SVM or a perceptron. 

* A Restricted Boltzmann Machine with binary visible units and binary hiddens. Parameters are estimated using Stochastic Maximum Likelihood (SML), also known as Persistent Contrastive Divergence (PCD)

* Two simple and easy search strategies for tuning parameter: Grid Search; Random Search

In [ ]:
from sklearn.neural_network import BernoulliRBM
from sklearn.cross_validation import train_test_split




### Gaussian Mixture Models with Universal Background Model
* A Gaussian mixture model is a probabilistic model that assumes all the data points are generated from a mixture of a finite number of Gaussian distributions wih unknown parameters. (Gaussian Distribution = Normal Distribution)
* The GMM object implements the EM algorithm for fitting mixture-of-Gaussian models. 
* GMM.fit method learns a Gaussian Mixture Model from train data. Gmm.predict method can assign each sample of test data to the Gaussian class which it mostly probably belong to.
* The GMM has different options to constrain the covariance: spherical, diagonal, tied or full covariance. Since we have class labels for the training data, we can initialize the GMM parameters in a supervised manner.

In [62]:
from sklearn.mixture import GMM
from sklearn.cross_validation import StratifiedKFold

#Try GMMs with different types of covariances
classifiers = dict((covar_type, GMM(n_components=n_classes, covariance_type=covar_type, init_params='wc', n_iter=20)) 
                   for covar_type in ['spherical', 'diag', 'tied', 'full']);

n_classifiers = len(classifiers);
print n_classifiers;
print classifiers.get('spherical');
plt.figure(figsize=(3 * n_classifiers / 2, 6));

# gmm = GMM(n_components=51);
# print gmm;
# gmm.fit(train_data);

4
GMM(covariance_type='spherical', init_params='wc', min_covar=0.001,
  n_components=51, n_init=1, n_iter=20, params='wmc', random_state=None,
  thresh=None, tol=0.001, verbose=0)


In [59]:
print test_data.shape;
print gmm.score(test_data[0:10]);
print gmm.predict(test_data[3000:3020]);
print test_target[3000:3020];

(4080L, 31L)
[ 41.01847567  32.96404447  43.91977852  39.8148304   47.09648183
  40.46836094  36.39326048  46.08843417  44.37268799  46.80432982]
[40 24 24 24 24 15 24 24 15 24 24 24 24 24 24 47 24 24 24 24]
[['s043']
 ['s043']
 ['s043']
 ['s043']
 ['s043']
 ['s043']
 ['s043']
 ['s043']
 ['s043']
 ['s043']
 ['s043']
 ['s043']
 ['s043']
 ['s043']
 ['s043']
 ['s043']
 ['s043']
 ['s043']
 ['s043']
 ['s043']]
